<a href="https://colab.research.google.com/github/bisht-deepak/webscrapping-Georgia-Tech-University/blob/main/georgia_tech_faculty_basic_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
#important links
georgiaTechBaseURL= "https://www.ece.gatech.edu/faculty-staff-directory"
landingPageURL= "?field_group_filter_value=1"

In [ ]:
#preparing soup
with requests.Session() as s:
    root= s.get(georgiaTechBaseURL+landingPageURL)
    soup = BeautifulSoup(root.text, "html.parser")

In [ ]:
grandList= []
for i in range(0, 129, 1):
    d= {}
    
    name= soup.find_all("div", {"class": "field-full-name"})[i]
    d["name"]= name.find("a").text
    
    title= soup.find_all("div", {"class": "field-jobtitle"})[i]
    d["jobtitle"]= title.text.replace("\n", "").replace(" ", "")

    mailid= soup.find_all('a', attrs={"href": re.compile("^mailto:")})[i]
    d["email_address"]= mailid.get('href')[7:]
        
    phone= soup.find_all("div", {"class": "field-work-phone"})[i]
    phone= phone.text.replace("\n", "").replace(" ", "")
    if phone== "":
        d["work_phone"]= None
    else:
        d["work_phone"]= phone
    
    #extract office address
    try:
        office= soup.find_all("div", {"class": "faculty-staff-content"})[i]
        d["office_address"]= office.find("div", {"class": "field-office"}).text.replace("\n", "").replace(" ", "") 
        #problem faced- total listings= 129, total offices= 116
    except:
        d["office_address"]= None   
    grandList.append(d)

In [ ]:
dF= pd.DataFrame(grandList)

In [ ]:
dF.head(5)

,name,jobtitle,email_address,work_phone,office_address
0,Chaouki Tanios Abdallah,Professor;ExecVP-ResearchatGeorgiaTech,ctabdallah@gatech.edu,None,None
1,Ali Adibi,Professor,ali.adibi@ece.gatech.edu,404/385-2738,BH105
2,Ghassan AlRegib,Professor,alregib@gatech.edu,404/894-7005,Cent5224
3,David V Anderson,Professor,anderson@gatech.edu,404/385-4979,TSRB543
4,Azadeh Ansari,AssistantProfessor,azadeh.ansari@ece.gatech.edu,None,TSRB544


In [ ]:
dF.to_csv("Georgia Tech Landing Page.csv")